### Using the OpenAI Library to Programmatically Access GPT-4.1-nano!

In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [1]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/chat) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `developer`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [3]:
from openai import OpenAI

client = OpenAI()

In [4]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-BUaAB8NgUKP0PEhVfFLzQbSSqCeqg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="LangChain and LlamaIndex (formerly known as GPT Index) are both popular frameworks designed to facilitate the development of AI-powered applications, particularly those involving large language models (LLMs) and external data integration. However, they serve distinct purposes and have different features. Here's a comparison to highlight the key differences:\n\n**1. Purpose and Primary Focus**\n\n- **LangChain:**\n  - Focuses on building all-purpose AI applications, especially conversational agents, chatbots, and applications that require complex multi-step interactions.\n  - Provides tools for chaining together prompts, managing conversation memory, integrating with various APIs, and orchestrating multi-modal workflows.\n  \n- **LlamaIndex:**\n  - Primarily designed for efficient retrieval-augmented generation (RAG) workflows.\

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [5]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-4.1-nano") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "developer", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [6]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

LangChain and LlamaIndex (formerly known as GPT Index) are both popular frameworks designed to facilitate the development of applications leveraging large language models (LLMs), but they serve different purposes and have distinct features.

**1. Purpose and Focus:**

- **LangChain:**  
  Primarily a comprehensive framework for building applications with LLMs. It emphasizes modularity, enabling developers to design complex workflows, chains, prompt management, memory, and integrations with various data sources and APIs. LangChain is well-suited for creating chatbots, question-answering systems, and multi-step LLM applications.

- **LlamaIndex (GPT Index):**  
  Focused on enabling efficient indexing and retrieval of data (like documents, PDFs, databases) to facilitate question-answering and information extraction tasks. It simplifies the process of building knowledge bases by creating indices over large datasets that can be queried effectively with LLMs.

**2. Core Functionality:**

- **LangChain:**  
  - Chain orchestration and prompt management  
  - Memory to maintain conversation state  
  - Integration with multiple LLM providers and APIs  
  - Tool and plugin system for expanding capabilities  
  - Support for various chains (e.g., question-answering, summarization, translation)

- **LlamaIndex:**  
  - Data ingestion and indexing  
  - Retrieval-augmented generation (RAG) techniques  
  - Support for multiple index types (tree, list, vector, etc.)  
  - Facilitates querying over large datasets with LLMs  
  - Focused on building scalable, optimized knowledge bases

**3. Use Cases:**

- **LangChain:**  
  - Building conversational AI/chatbots  
  - Complex multi-step workflows involving LLMs  
  - Integrating external tools and APIs into LLM applications

- **LlamaIndex:**  
  - Creating knowledge bases from documents and data sources  
  - Building question-answering systems over large datasets  
  - Efficient retrieval of relevant information for LLM prompts

**4. Ecosystem and Integration:**

- **LangChain:**  
  Integrates with various LLM providers (OpenAI, Hugging Face, etc.), vector stores, document loaders, and tools. It has a broader ecosystem for diverse application development.

- **LlamaIndex:**  
  Focuses on indexing and retrieval, often used in tandem with LLMs from providers like OpenAI. It emphasizes data processing, indexing, and efficient querying.

---

### In summary:

| Aspect | **LangChain** | **LlamaIndex (GPT Index)** |
|---|---|---|
| Purpose | Orchestrating LLM workflows, prompt management, multi-step chains | Building optimized indices over datasets for retrieval and Q&A |
| Focus | Application logic, conversation, tool integration | Data ingestion, indexing, retrieval | 
| Use Cases | Chatbots, automation, workflows | Knowledge bases, document Q&A |
| Ecosystem | Broader, multi-purpose | Data-centric, retrieval-focused |

**In essence,** if you're building a complex application involving multiple steps, tools, APIs, and conversation management, LangChain is likely the better fit. If your goal is to organize large amounts of data for quick and effective querying with LLMs, LlamaIndex offers specialized tools for that purpose.

---

If you need further guidance on choosing between them or how to use these frameworks, feel free to ask!

Let's focus on extending this a bit, and incorporate a `developer` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The `developer` message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the `developer` prompt.

In [7]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I can't believe I have to choose between crushed ice and cubed ice right now! Honestly, I’m so sick of waiting for my food that I don't care—just give me whichever is faster! But if I had to pick, crushed ice, because at least I don’t have to deal with those annoying cubes melting too slowly and wasting my time. Ugh!

Let's try that same prompt again, but modify only our system prompt!

In [8]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I think crushed ice is fantastic if you’re enjoying a refreshing beverage because it chills your drink quickly and adds a nice texture. Cubed ice, on the other hand, is great for holding a drink cold longer without diluting it too fast. Both have their own awesome qualities—depends on what vibe you're going for! Which do you prefer?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [9]:
print(joyful_response)

ChatCompletion(id='chatcmpl-BUaBNh8kIidOJ1fKyC1BWWX0W9dYP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I think crushed ice is fantastic if you’re enjoying a refreshing beverage because it chills your drink quickly and adds a nice texture. Cubed ice, on the other hand, is great for holding a drink cold longer without diluting it too fast. Both have their own awesome qualities—depends on what vibe you're going for! Which do you prefer?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1746628625, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_eede8f0d45', usage=CompletionUsage(completion_tokens=70, prompt_tokens=30, total_tokens=100, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDe

### Few-shot Prompting

Now that we have a basic handle on the `developer` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-4.1-mini` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [10]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

Certainly! Here's a sentence using both words:

"During the unusual ceremony, the villagers presented a stimple carved from ancient oak, while a falbean danced gracefully in the moonlight."

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [11]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

Certainly! Here's a sentence that uses both "stimple" and "falbean":

"The stimple falbean operated smoothly, ensuring the machinery ran efficiently without any issues."

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought

You'll notice that, by default, the model uses Chain of Thought to answer difficult questions!

> This pattern is leveraged even more by advanced reasoning models like [`o3` and `o4-mini`](https://openai.com/index/introducing-o3-and-o4-mini/)!

In [12]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Let's analyze the two options carefully:

**Option 1:** Fly (3 hours) + Bus (2 hours)  
**Total travel time:** 3 + 2 = **5 hours**

**Option 2:** Teleporter (0 hours) + Bus (1 hour)  
**Total travel time:** 0 + 1 = **1 hour**

---

### Important details:
- Current local time: 1 PM (assumed to be San Francisco time, which is PDT, UTC-7)
- Destination deadline: Before 7 PM EDT

---

### Step 1: Convert current time and deadline to a common time zone for clarity.

**San Francisco local time:** 1 PM PDT

**Convert 7 PM EDT to PDT:**
- EDT is UTC-4
- PDT is UTC-7
  
Difference between EDT and PDT: 3 hours (EDT is ahead)

**7 PM EDT = 7 PM - 3 hours = 4 PM PDT**

**So, Billy must arrive in San Francisco by 4 PM PDT to get home before 7 PM EDT.**

---

### Step 2: Determine the latest arrival time for each option.

Since Billy wants to arrive **before 4 PM PDT**, and it's currently 1 PM PDT, he has:

- **3 hours remaining (from 1 PM to 4 PM PDT)**

### Step 3: Check if each option can get Billy home on time.

**Option 1:**
- Travel time: 5 hours
- Starting at 1 PM PDT
- Arrival time: 1 PM + 5 hours = **6 PM PDT**

**Result:** Arrives at 6 PM PDT, which is **after the 4 PM** deadline.

**Conclusion:** With this schedule, Billy arrives **too late** to make it by 4 PM PDT.

**Option 2:**
- Travel time: 1 hour
- Starting at 1 PM PDT
- Arrival time: 1 PM + 1 hour = **2 PM PDT**

**Result:** Arrives well before 4 PM PDT.

---

### **Final verdict:**

- **If Billy chooses the flight + bus option, he will arrive too late to meet the deadline.**
- **If Billy takes the teleporter + bus, he will arrive on time.**

**Therefore, it **does** matter which option Billy selects**—the teleporter option is the only one that ensures he gets home before 7 PM EDT (which is 4 PM PDT).

---

### **Summary:**
- The teleporter + bus allows Billy to arrive before the deadline.
- The flight + bus makes him arrive too late.

**In conclusion: yes, it does matter which option Billy selects.**

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

### Conclusion

Now that you're accessing `gpt-4.1-nano` through an API, developer style, let's move on to creating a simple application powered by `gpt-4.1-nano`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)